# Inferenza

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

# Training

### Importazione dell'immagine da ecr

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  763104351884.dkr.ecr.us-east-1.amazonaws.com

### Build dell'immagine

Attraverso Dockerfile.train a partire dalla immagine di ECR

In [ ]:
!docker build -t 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-housing -f Dockerfile.train .

In [ ]:
!docker images

## Training in locale

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-housing',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd()),
)

print('##### ESTIMATOR FIT STARTED')
print(os.getcwd())
input_path = 'file://{}/data/input/'.format(os.getcwd())
estimator.fit(input_path+"train.csv")
print('##### ESTIMATOR FIT COMPLETED')

### Spacchettamento output

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!tar -xzf ./data/output/model.tar.gz

## Training in remoto (SageMaker)

### Calcolo del full name (URI) da dare all'immagine

In [ ]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-housing"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing

In [ ]:
!docker tag 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-housing 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing 

In [ ]:
!docker push 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsarghisdata/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsarghisdata/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')



In [ ]:
from